# Data Preperation

In [86]:
from datetime import date, datetime, timedelta, time
import holidays
import pytz
import pandas as pd
import numpy as np

## Functions

In [87]:
def removeTzRows(data, dtList): 
    """remove rows of datetimes that do not exist due to tz changes."""
    for d in dtList:
        if d in data.index:
            data.drop(d, inplace=True)
            print(str(d) + " dropped.")  
    return data
def newIndex(start, end, delta):
    """create a new index for reindexing."""
    curr = start
    while curr < end:
        yield curr
        curr += delta
        
def prepData(data, delta=60):
    # some zones have missing values
    # that are supposed to be zero.
    data.fillna(0, axis=1, inplace=True)
    # drop the hour between 2 and 3 
    # when US time goes so summer time
    data = removeTzRows(data, tzDates)
    # drop duplicate index rows
    duplicateNumber = data.index.duplicated().astype(int).sum()
    data = data.loc[~data.index.duplicated(keep='first')]
    print(str(duplicateNumber) + " duplicates found and removed.")
    # create new index with 30 minute time interval
    reindx = list()
    for result in newIndex(startDt, endDt, timedelta(minutes=delta)):
        reindx.append(result)
    # reindex so there are no missing datetimes
    data = data.reindex(reindx)
    # fill missing values with their previous one
    data.fillna(method="ffill", inplace=True)
    return data 

## Constants

In [88]:
startDt = datetime(2017, 1, 1, 0, 0)
endDt = datetime(2020, 1, 1, 0, 0)
# datetimes that should not exist due to tz changes
tzDates = [datetime(2017,3,12,2,0), datetime(2017,3,12,2,30),
            datetime(2018,3,11,2,0), datetime(2018,3,11,2,30),
            datetime(2019,3,10,2,0), datetime(2019,3,10,2,30)
           ]
usHolidays = holidays.UnitedStates()

In [89]:
#load zone details
zones = pd.read_csv('taxi_data/taxiZoneLookup.csv')

## Aggregated Data

* aggregates trips for every dt for each zone
* time-series format
* used to explore trips more easily

In [128]:
def get_aggr_data(years):
    check = 1
    for year in years:
        for month in range(1,13):
            path = "taxi_data/yellow_tripdata_{}-{}{}".format(year, f"{month:02d}", ".csv")
            df = pd.read_csv(path, 
                             parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"], 
                             low_memory=False)
            
            df = df[["tpep_pickup_datetime", 
                     "tpep_dropoff_datetime", 
                     "trip_distance", 
                     "passenger_count", 
                     "PULocationID", 
                     "DOLocationID"]]
            
            #only trips shorter than 100 miles
            df = df[df.trip_distance < 100]
            #only trips with at least 1 passenger
            df = df[df.passenger_count != 0]
            #calculate trip duration
            df["trip_duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
            #only trips longer than 1 minute and shorter than 3 hours
            df = df[(df.trip_duration > timedelta(minutes=1)) & (df.trip_duration < timedelta(hours=3))]
            
            #merge borough from the zones dataframe
            #so trips can be filtered for Manhattan, Queens and Brooklyn only
            df.rename(columns={"PULocationID": "LocationID"}, inplace=True)
            df = pd.merge(df, zones[["LocationID", "Borough"]], on="LocationID", how="left")
            df.rename(columns={"LocationID": "PULocationID", "Borough": "PUBorough", "Zone": "PUZone"}, inplace=True)
            df.rename(columns={"DOLocationID": "LocationID"}, inplace=True)
            df = pd.merge(df, zones[["LocationID", "Borough"]], on="LocationID", how="left")
            df.rename(columns={"LocationID": "DOLocationID", "Borough": "DOBorough", "Zone": "DOZone"}, inplace=True)
            
            #only include trips from and to Manhattan, Queens or Brooklyn
            boroughs = ["Manhattan", "Queens", "Brooklyn"]
            df = df[(df.PUBorough.isin(boroughs)) & (df.DOBorough.isin(boroughs))]
            
            #drop everything except datetime information and pickup location
            df = df[["tpep_pickup_datetime", "PULocationID"]]
            
            #only include trips that fill in the current year and month
            if month != 12:
                df = df[(df.tpep_pickup_datetime >= datetime(year,month,1)) & (df.tpep_pickup_datetime < datetime(year,month+1,1))]
            else:
                df = df[(df.tpep_pickup_datetime >= datetime(year,month,1)) & (df.tpep_pickup_datetime < datetime(year+1,1,1))]
            
            #round pickup datetime to full hours
            df["tpep_pickup_datetime"] = df["tpep_pickup_datetime"].apply(lambda x: x - timedelta(minutes=x.minute % 60, seconds=x.second))    
            #create a dummy for each zone
            df = df[["tpep_pickup_datetime"]].join(pd.get_dummies(df.PULocationID))
            #aggregate trips for every zone at every pickup datetime
            df = df.groupby("tpep_pickup_datetime").sum()    
            
            #aggregate data unless it's the first data set (here 1/2017)
            if check == 0:
                final = pd.concat([final, df], sort=False)
            else:
                final = df
                check = 0
                
            print(str(year) + "-" + str(month))
            
    return final
            

In [130]:
#years = [2017,2018,2019]
years = [2019]
df1 = get_aggr_data(years)

2019-1
2019-2
2019-3
2019-4
2019-5
2019-6
2019-7
2019-8
2019-9
2019-10
2019-11
2019-12


In [131]:
df1.to_csv("taxi_data/aggr_taxi_data.csv")

In [71]:
#df1 = pd.read_csv("taxi_data/aggr_taxi_data.csv", index_col=["pickup_dt"], parse_dates=["pickup_dt"])

In [76]:
#remove duplicates or dts that do not exist due to summertime/wintertime shifts
df1 = prepData(df1)

2017-03-12 02:00:00 dropped.
2018-03-11 02:00:00 dropped.
0 duplicates found and removed.


In [77]:
#df1.to_csv("taxi_data/aggr_taxi_data.csv")

## Data for Forecasts

* raw data which will be the base for predictions later
* data is accumulated such that every dt has an aggregated number of trips for each zone
* no time-series format, but all zones can be included in one model

In [107]:
def get_data(years):
    check = 1
    for year in years:
        #print(year)
        for month in range(1,13):
            date_str = str(year)+"-"+str(month)
            #print(date_str)
            
            path = "taxi_data/yellow_tripdata_{}-{}{}".format(year, f"{month:02d}", ".csv")
            df = pd.read_csv(path, 
                             parse_dates=["tpep_pickup_datetime", 
                                          "tpep_dropoff_datetime"], 
                             #index_col=["tpep_pickup_datetime"], 
                             #low_memory=False
                            )
                     
            start = datetime(year,month,1)
            if month != 12:
                end = datetime(year,month+1,1)
            else:
                end = datetime(year+1,1,1)
            df = df[(df.tpep_pickup_datetime>=start) & (df.tpep_pickup_datetime<end)]
            df = df[["tpep_pickup_datetime",
                     "tpep_dropoff_datetime",
                     "trip_distance", 
                     "passenger_count",
                     "PULocationID",
                     "DOLocationID"]]
            
            df = df[df.trip_distance < 100]
            df = df[df.passenger_count != 0]
            df["trip_duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
            df = df[(df.trip_duration > timedelta(minutes=1)) & (df.trip_duration < timedelta(hours=3))]
            
            #merge borough from the zones dataframe
            #so trips can be filtered for Manhattan, Queens and Brooklyn only
            df.rename(columns={"PULocationID": "LocationID"}, inplace=True)
            df = pd.merge(df, zones[["LocationID", "Borough"]], on="LocationID", how="left")
            df.rename(columns={"LocationID": "PULocationID", 
                               "Borough": "PUBorough", 
                               "Zone": "PUZone"}, 
                      inplace=True)
            
            df.rename(columns={"DOLocationID": "LocationID"}, inplace=True)
            df = pd.merge(df, zones[["LocationID", "Borough"]], on="LocationID", how="left")
            df.rename(columns={"LocationID": "DOLocationID", 
                               "Borough": "DOBorough", 
                               "Zone": "DOZone"}, 
                      inplace=True)
            
            #only include trips from and to Manhattan, Queens or Brooklyn
            boroughs = ["Manhattan", "Queens", "Brooklyn"]
            df = df[(df.PUBorough.isin(boroughs)) & (df.DOBorough.isin(boroughs))]
            
            df = df[["tpep_pickup_datetime", "PULocationID"]]
            df["year"] = df["tpep_pickup_datetime"].apply(lambda x: x.year)
            df["month"] = df["tpep_pickup_datetime"].apply(lambda x: x.month)
            df["day"] = df["tpep_pickup_datetime"].apply(lambda x: x.day)
            df["hour"] = df["tpep_pickup_datetime"].apply(lambda x: x.hour)
            #df["wday"] = df["tpep_pickup_datetime"].apply(lambda x: x.weekday())
            df = df.drop("tpep_pickup_datetime", axis=1)
            df["rides"] = 0
            df = df.groupby(["PULocationID", "year", "month", "day", "hour"]).agg("count")
            df.reset_index(inplace=True)  
            if check == 0:
                final = pd.concat([final, df], sort=False)
            else:
                final = df
                check = 0
            print(str(year) + "-" + str(month) + " done!")
    return final

In [111]:
#df2 = get_data(years)

In [115]:
df2[(df2.PULocationID==161) & (df2.year==2019) & (df2.month==12) & (df2.day==17)]

,PULocationID,year,month,day,hour,rides
53961,161,2019,12,17,0,282
53962,161,2019,12,17,1,103
53963,161,2019,12,17,2,49
53964,161,2019,12,17,3,26
53965,161,2019,12,17,4,20
53966,161,2019,12,17,5,41
53967,161,2019,12,17,6,110
53968,161,2019,12,17,7,286
53969,161,2019,12,17,8,411
53970,161,2019,12,17,9,461


In [110]:
df2.to_csv("taxi_data/taxi_data.csv")

## Weather Data

* adjust data from UTC to NYC time (including daylist saving time)

In [78]:
def loadWeatherData(path, tarTimezone="US/Eastern", dateFormat="%Y-%m-%d %H:%M"):
    """Read weather data, transform datetime to target timezone, set as index.
    Keyword arguments:
    path -- path of weather data with data type
    tarTimezone -- target timezone date is transformed into
    dateFormat -- format of datetime
    """
    w = pd.read_csv(path)
    eastern = pytz.timezone(tarTimezone)
    fmt = dateFormat
    w["dt_iso"] = w["dt_iso"].apply(lambda x: pd.to_datetime(x[:-10], utc=True))
    w["dt"] = w["dt_iso"].apply(lambda x: x.astimezone(eastern).strftime(fmt))
    w["dt"] = w["dt"].apply(lambda x: pd.to_datetime(x))
    w = w.set_index("dt")
    return w[["temp", "feels_like", "temp_min", "temp_max", "humidity", "wind_speed", 
              "rain_1h", "rain_3h", "snow_1h", "snow_3h", "weather_main", "weather_description"]]

In [80]:
w = loadWeatherData("taxi_data/weather.csv")

In [82]:
w = w.fillna(0)

In [84]:
w = prepData(w)

322 duplicates found and removed.


In [85]:
w.to_csv("taxi_data/weather_prepared.csv")